![image.png](https://raw.githubusercontent.com/fjvarasc/DSPXI/master/figures/py_logo.png)


<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#SQL,-ETL,-and-SQLAlchemy-in-Python" data-toc-modified-id="SQL,-ETL,-and-SQLAlchemy-in-Python-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>SQL, ETL, and SQLAlchemy in Python</a></span><ul class="toc-item"><li><span><a href="#Extract" data-toc-modified-id="Extract-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Extract</a></span></li><li><span><a href="#Transform" data-toc-modified-id="Transform-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Transform</a></span><ul class="toc-item"><li><span><a href="#Bringing-all-together" data-toc-modified-id="Bringing-all-together-5.2.1"><span class="toc-item-num">5.2.1&nbsp;&nbsp;</span>Bringing all together</a></span></li></ul></li><li><span><a href="#Load" data-toc-modified-id="Load-5.3"><span class="toc-item-num">5.3&nbsp;&nbsp;</span>Load</a></span></li></ul></li></ul></div>

# SQL, ETL, and SQLAlchemy in Python

SQL is simpler and more readable than Pandas, which is why many people prefer it. It's also fast, and it's the language for running queries of your database/data warehouses. This is the stuff of data science at scale!

The core component that this workflow involves is called ETL, which stands for Extract, Transform, Load. While this example is a local system project, if the database file(s) were sourced from a source system, extraction would involve moving it into a data warehouse. From there it would be transformed using SQL queries. The final step would be loading the data into something like Python and Pandas to do machine learning.

I used SQLAlchemy, which uses SQLite under the hood. SQLAlchemy has its own set of classes and methods for running SQL queries, but I wrote out raw SQL instead for readers who are more interested in seeing that or more familiar with that. I also did this to show the logic of my queries, since all that would be abstracted away by SQLAlchemy.

## Extract

In [35]:
# Import statements
import sqlalchemy
from sqlalchemy import create_engine,Table, Column, Integer, String, MetaData, ForeignKey,inspect

# Connect the engine to the database file we'll be using
engine = create_engine('sqlite:///chinook.db')
engine

Engine(sqlite:///chinook.db)

Here is the chinook database schema. We are going to start with just the employees data table.

![image](https://github.com/fjvarasc/DSPXI/raw/master/figures/sqlite-sample-database-color.png)


In [36]:
# SQL Expression Language creates metadata that contains objects that define the customers table
metadata = MetaData()

# This method instantiates the tables that already 
# exist in the database, which the engine is connected to. 
metadata.create_all(engine)

# Checking this out, we can see the table structure and variable types for the employees table
inspector = inspect(engine)

# Checked out the columns in the employees table
inspector.get_columns('employees')

[{'autoincrement': 'auto',
  'default': None,
  'name': 'EmployeeId',
  'nullable': False,
  'primary_key': 1,
  'type': INTEGER()},
 {'autoincrement': 'auto',
  'default': None,
  'name': 'LastName',
  'nullable': False,
  'primary_key': 0,
  'type': NVARCHAR(length=20)},
 {'autoincrement': 'auto',
  'default': None,
  'name': 'FirstName',
  'nullable': False,
  'primary_key': 0,
  'type': NVARCHAR(length=20)},
 {'autoincrement': 'auto',
  'default': None,
  'name': 'Title',
  'nullable': True,
  'primary_key': 0,
  'type': NVARCHAR(length=30)},
 {'autoincrement': 'auto',
  'default': None,
  'name': 'ReportsTo',
  'nullable': True,
  'primary_key': 0,
  'type': INTEGER()},
 {'autoincrement': 'auto',
  'default': None,
  'name': 'BirthDate',
  'nullable': True,
  'primary_key': 0,
  'type': DATETIME()},
 {'autoincrement': 'auto',
  'default': None,
  'name': 'HireDate',
  'nullable': True,
  'primary_key': 0,
  'type': DATETIME()},
 {'autoincrement': 'auto',
  'default': None,
  'name

## Transform

In [110]:
# Let's execute raw SQL on some tables using SQLAlchemy
with engine.connect() as con:
    
    rs = con.execute('SELECT * FROM employees')
    
    for row in rs:
        print(row)
        
# Don't forget to close your connection to the database when the query is done
con.close()

# Note: Use LIMIT n to limit the number of rows returned. Not a problem with this small table

(1, 'Adams', 'Andrew', 'General Manager', None, '1962-02-18 00:00:00', '2002-08-14 00:00:00', '11120 Jasper Ave NW', 'Edmonton', 'AB', 'Canada', 'T5K 2N1', '+1 (780) 428-9482', '+1 (780) 428-3457', 'andrew@chinookcorp.com')
(2, 'Edwards', 'Nancy', 'Sales Manager', 1, '1958-12-08 00:00:00', '2002-05-01 00:00:00', '825 8 Ave SW', 'Calgary', 'AB', 'Canada', 'T2P 2T3', '+1 (403) 262-3443', '+1 (403) 262-3322', 'nancy@chinookcorp.com')
(3, 'Peacock', 'Jane', 'Sales Support Agent', 2, '1973-08-29 00:00:00', '2002-04-01 00:00:00', '1111 6 Ave SW', 'Calgary', 'AB', 'Canada', 'T2P 5M5', '+1 (403) 262-3443', '+1 (403) 262-6712', 'jane@chinookcorp.com')
(4, 'Park', 'Margaret', 'Sales Support Agent', 2, '1947-09-19 00:00:00', '2003-05-03 00:00:00', '683 10 Street SW', 'Calgary', 'AB', 'Canada', 'T2P 5G3', '+1 (403) 263-4423', '+1 (403) 263-4289', 'margaret@chinookcorp.com')
(5, 'Johnson', 'Steve', 'Sales Support Agent', 2, '1965-03-03 00:00:00', '2003-10-17 00:00:00', '7727B 41 Ave', 'Calgary', 'A

"SELECT * FROM table_name" is the hello world of SQL. Let's go a little more in depth.

In [111]:
# How many employees are there?
with engine.connect() as con:
    
    rs = con.execute("""SELECT COUNT(EmployeeId) 
    FROM employees;""")
    
    for row in rs:
        print(row)
        
con.close()

(8,)


In [115]:
# How many customers did each sales rep help?
with engine.connect() as con:
    
    rs = con.execute("""SELECT SupportRepId,COUNT(SupportRepId)
    FROM customers
    GROUP BY SupportRepId;""")
    
    for row in rs:
        print(row)
        
con.close()

(3, 21)
(4, 20)
(5, 18)


Employees 3, 4, and 5 helped 21, 20, and 18 customers respectively.

In [116]:
# Does their length of tenure map to how many customers they helped?
with engine.connect() as con:
    
    rs = con.execute("""SELECT MIN(HireDate), EmployeeId
    FROM employees;""")
    
    for row in rs:
        print(row)
        
con.close()

('2002-04-01 00:00:00', 3)


The most senior employee helped the most customers.

In [117]:
with engine.connect() as con:
    
    rs = con.execute("""SELECT HireDate, EmployeeId
    FROM employees
    WHERE EmployeeId BETWEEN 3 AND 5
    ORDER BY HireDate ASC""")
    
    for row in rs:
        print(row)

('2002-04-01 00:00:00', 3)
('2003-05-03 00:00:00', 4)
('2003-10-17 00:00:00', 5)


Length of tenure in the company seems to be somewhat linear. The most senior sales rep helped the most customers, and the most junior rep helped the least.

### Bringing all together

In a relational database, there are relations between tables. Look back up at the database schema above. The customers table for example has as its primary key the CustomerId. This is a unique identifier for each customer. While there could be more than one John, there wouldn't be more than one CustomerId number 2355.

How many minutes of music was listened to for each country? How many customers were there for each country?

To answer these questions, we need to get to the tracks table. The customers table tells us CustomerId and Country, so we can count CustomerId to derive the number of customers. 

To get to the Milliseconds variable in tracks though, we need to join that table with TrackId in invoice_items, InvoiceId with the invoices table, and finally the CustomerId in  the customers table. That's our roadmap.

In [118]:
with engine.connect() as con:
    
    # Grab the variables you want then inner join them on the respective private keys
    rs = con.execute(
    """SELECT
    invoices.InvoiceId AS invid, 
    invoices.CustomerId AS invcustid, 
    customers.CustomerId AS custcustid,
    COUNT(customers.CustomerId) AS numcustomers,
    customers.Country as country,
    invoice_items.InvoiceId AS invitemid,
    
    invoice_items.TrackId AS invtrackid,
    tracks.TrackId AS tracktrackid, 
    tracks.GenreId AS trackgenreid, 
    tracks.Bytes AS trackbytes, 
    SUM(tracks.Milliseconds) / 1000 / 60 AS minutes
    
    FROM 
    invoices INNER JOIN customers ON invcustid=custcustid
    INNER JOIN invoice_items ON invitemid=invid
    INNER JOIN tracks ON tracktrackid=invtrackid
    
    GROUP BY country
    
    ORDER BY minutes DESC
    
    """
    )
    
    for row in rs:
        print(row)

con.close()

(363, 28, 28, 494, 'USA', 363, 1553, 1553, 3, 9962558, 3277)
(388, 33, 33, 304, 'Canada', 388, 2287, 2287, 4, 7373181, 1438)
(368, 43, 43, 190, 'France', 368, 1645, 1645, 1, 7236640, 1168)
(291, 38, 38, 152, 'Germany', 291, 2596, 2596, 4, 6399872, 914)
(319, 13, 13, 190, 'Brazil', 319, 21, 21, 1, 8331286, 881)
(404, 6, 6, 76, 'Czech Republic', 404, 2931, 2931, 1, 11527614, 879)
(314, 57, 57, 38, 'Chile', 314, 3432, 3432, 24, 8916313, 525)
(381, 54, 54, 114, 'United Kingdom', 381, 2055, 2055, 7, 5857116, 506)
(401, 46, 46, 38, 'Ireland', 401, 2727, 2727, 4, 9601786, 476)
(410, 35, 35, 76, 'Portugal', 410, 3037, 3037, 1, 9258717, 429)
(284, 59, 59, 74, 'India', 284, 2364, 2364, 4, 7971384, 417)
(377, 45, 45, 38, 'Hungary', 377, 2017, 2017, 1, 7354184, 380)
(370, 7, 7, 38, 'Austria', 370, 1785, 1785, 14, 8888206, 374)
(411, 44, 44, 38, 'Finland', 411, 3163, 3163, 7, 5121366, 355)
(390, 48, 48, 38, 'Netherlands', 390, 2467, 2467, 1, 8222095, 305)
(392, 4, 4, 38, 'Norway', 392, 2483, 2483, 

The US customers rocked out with 3,277 minutes, or 54.6 hours!

## Load

In [126]:
# Connecting the query to pd.read_sql_query. To simplify, you could modify the query to create
# a table and then just do pd.read_sql_table in to the dataframe.
import pandas as pd
df = pd.read_sql_query("""SELECT
    invoices.InvoiceId AS invid, 
    invoices.CustomerId AS invcustid, 
    customers.CustomerId AS custcustid,
    COUNT(customers.CustomerId) AS numcustomers,
    customers.Country as country,
    invoice_items.InvoiceId AS invitemid,
    
    invoice_items.TrackId AS invtrackid,
    tracks.TrackId AS tracktrackid, 
    tracks.GenreId AS trackgenreid, 
    tracks.Bytes AS trackbytes, 
    SUM(tracks.Milliseconds) / 1000 / 60 AS minutes
    
    FROM 
    invoices INNER JOIN customers ON invcustid=custcustid
    INNER JOIN invoice_items ON invitemid=invid
    INNER JOIN tracks ON tracktrackid=invtrackid
    
    GROUP BY country
    
    ORDER BY minutes DESC
    
    """, con=engine.connect())

In [120]:
df.head()

,invid,invcustid,custcustid,numcustomers,country,invitemid,invtrackid,tracktrackid,trackgenreid,trackbytes,minutes
0,363,28,28,494,USA,363,1553,1553,3,9962558,3277
1,388,33,33,304,Canada,388,2287,2287,4,7373181,1438
2,368,43,43,190,France,368,1645,1645,1,7236640,1168
3,291,38,38,152,Germany,291,2596,2596,4,6399872,914
4,319,13,13,190,Brazil,319,21,21,1,8331286,881
